# SAS 関連

## datetime について

import 増やしてコード量減らすか、逆かどっちがいいかね...と悩む。

In [ ]:
%pip install -r requirements.txt

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# import 少ない ver
import datetime

start_time = datetime.datetime.now(datetime.timezone.utc) - datetime.timedelta(minutes=15)
expiry_time = start_time + datetime.timedelta(days=1)

print(start_time)
print(expiry_time)

2023-11-10 02:13:19.644204+00:00
2023-11-11 02:13:19.644204+00:00


In [5]:
# import 多い ver
from datetime import datetime, timedelta, timezone

start_time = datetime.now(timezone.utc) - timedelta(minutes=15)
expiry_time = start_time + timedelta(days=1)

print(start_time)
print(expiry_time)


2023-11-10 02:11:18.557440+00:00
2023-11-11 02:11:18.557440+00:00


## Blob の SAS URL 取得

AccountKey が単独で必要になるのは注意。

In [15]:
from datetime import datetime, timedelta, timezone
import os
from azure.storage.blob import BlobClient, BlobSasPermissions, generate_blob_sas


print(start_time)
print(expiry_time)


def get_blob_url_with_sas(blob_client: BlobClient):
    # https://learn.microsoft.com/ja-jp/azure/storage/blobs/sas-service-create-python#create-a-service-sas-for-a-blob
    
    connection_str = os.environ["BLOB_CONNECTION_STRING"]
    
    # 必要な情報は以下。
    account_key = connection_str.split(';')[2].split('AccountKey=')[1]
    blob_account_name = blob_client.account_name
    blob_container_name = blob_client.container_name
    blob_name = blob_client.blob_name
    
    # 有効期限は15分前から1日間の場合
    start_time = datetime.now(timezone.utc) - timedelta(minutes=15)
    expiry_time = start_time + timedelta(days=1)

    sas_token = generate_blob_sas(
        account_name=blob_account_name,
        container_name=blob_container_name,
        blob_name=blob_name,
        account_key=account_key,
        permission=BlobSasPermissions(read=True),
        expiry=expiry_time,
        start=start_time
    )

    blob_url_with_sas = f"{blob_client.url}?{sas_token}"
    print(f"{blob_url_with_sas}")


blob_client = BlobClient.from_connection_string(os.environ["BLOB_CONNECTION_STRING"], "test1", "test.txt")
get_blob_url_with_sas(blob_client)

2023-11-10 02:13:19.644204+00:00
2023-11-11 02:13:19.644204+00:00
https://styokosandbox.blob.core.windows.net/test1/test.txt?st=2023-11-10T02%3A36%3A19Z&se=2023-11-11T02%3A36%3A19Z&sp=r&sv=2023-11-03&sr=b&sig=SsHgwRpe/%2B49lAsKiPncvFAxs/HTMzuIgTAbmcYzB8w%3D
